In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
dfp= pd.read_csv('../input/quantium-data-analytics-virtual-experience-program/PurchaseBehaviour.csv')
dft= pd.read_csv('../input/quantium-data-analytics-virtual-experience-program/Transactions.csv')

In [ ]:
dfp.head()

In [ ]:
dft.head()

In [ ]:
len(dft)-len(dft.drop_duplicates())

There is one duplicate. We will remove it

In [ ]:
len(dfp)-len(dfp.drop_duplicates())

No duplicates in this table

In [ ]:
dft_unique=dft.drop_duplicates()

In [ ]:
dft_unique.describe()

In [ ]:
dfp.describe()

In [ ]:
dft_unique.isna().sum()

In [ ]:
dfp.isna().sum()

#### convert date to date format from int



In [ ]:
base_date = pd.Timestamp('1901-01-01')  
dates = [
    base_date + pd.DateOffset(dft['DATE'][i]) 
    for i in range(len(dft['DATE']))
]

print(dates[0:5])

In [ ]:
dft['DATE']=dates
dft['DATE'].head()

 Converting Date time to just Time 

In [ ]:
dft['DATE']=dft['DATE'].dt.date

In [ ]:
dft['DATE'].head()

No Empty values and no duplicates. We can merge the dataset now. 

In [ ]:
noncommon = dft_unique[~dft_unique.LYLTY_CARD_NBR.isin(dfp.LYLTY_CARD_NBR)]

In [ ]:
len(noncommon)

In [ ]:
merged_df = pd.merge(left=dfp, right=dft, how='outer', left_on='LYLTY_CARD_NBR', right_on='LYLTY_CARD_NBR')

In [ ]:
merged_df.head()

In [ ]:
merged_df.tail()

In [ ]:
merged_df.shape

In [ ]:
merged_df.isna().sum()

In [ ]:
import re

In [ ]:
def _removeNonAscii(s):
    return "".join(i for i in s if  ord(i)<128)
def punctuation(s):
    return re.sub(r'[^\w\s]', '', s)

In [ ]:
merged_df['PROD_NAME']=merged_df['PROD_NAME'].apply(_removeNonAscii)
merged_df['PROD_NAME']=merged_df['PROD_NAME'].apply(punctuation)

In [ ]:
merged_df['PROD_NAME_1'] = merged_df['PROD_NAME'].str.replace('\d+','')

In [ ]:
from collections import Counter
Counter(" ".join(merged_df['PROD_NAME_1']).split()).most_common(50)

'g' is the most common because it appears in every product to indicate it's size

In [ ]:
merged_df['Last'] = merged_df['PROD_NAME'].str.strip().str[-1]

In [ ]:
merged_df['Last'].value_counts()

In [ ]:
merged_df['PROD_SIZE']=merged_df['PROD_NAME'].str.extract('(\d+)')

In [ ]:
merged_df.PROD_SIZE=merged_df.PROD_SIZE.astype('int64')

In [ ]:
merged_df=merged_df.drop('PROD_NAME',axis=1)

In [ ]:
chip_words=['Chip','Chips','Tortilla','Crisp','Twisties']

In [ ]:
merged_df['PROD_QTY'].min()

In [ ]:
merged_df['PROD_QTY'].max()

In [ ]:
merged_df[merged_df['PROD_QTY']==200]

They're both bought by the same customer. Looks like a wholesale customer. We can omit this data

In [ ]:
merged_df =merged_df[merged_df['PROD_QTY'] !=200]

In [ ]:
def convertsize(number):
    if number < 100:
        return 'Small'
    if number >= 100 and number < 200:
        return 'Medium'
    if number >= 200:
        return 'Large'

In [ ]:
merged_df['PROD_SIZE']=merged_df['PROD_SIZE'].apply(convertsize)

In [ ]:
merged_df.groupby(by='PROD_SIZE')['PREMIUM_CUSTOMER'].value_counts()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(rc={'figure.figsize':(20, 20)})


In [ ]:
date_df=merged_df


In [ ]:
date_df.set_index('DATE')

In [ ]:
sns.set(rc={'figure.figsize':(10, 5)})
date_df.groupby('LIFESTAGE').TOT_SALES.sum().plot(kind='bar')

In [ ]:
sns.set(rc={'figure.figsize':(10, 5)})
date_df.groupby('PREMIUM_CUSTOMER').TOT_SALES.sum().plot(kind='bar')

In [ ]:
pd.pivot_table(merged_df, values='TOT_SALES', index=['PREMIUM_CUSTOMER'],
                    columns=['LIFESTAGE'], aggfunc=np.sum)

In [ ]:
merged_df.groupby(by='PREMIUM_CUSTOMER')['PROD_NAME_1'].value_counts()

In [ ]:
fig, ax = plt.subplots(figsize=(20,8))
sns.histplot(binwidth=0.5, x="LIFESTAGE", hue="PREMIUM_CUSTOMER", data=merged_df, stat="count", multiple="stack")

The highest proportion of mainstream customers come from young families, which could be because they earn not too much and might be just starting out in careers. 

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

date_df.groupby('DATE').TOT_SALES.sum().plot(kind='line')

Sales peak during December. This could be because of Christmas.
